# Projeto 3
## André Ejzenmesser
## Vinicius Bonelli

Variáveis escolhidas:

Variável resposta: Indice de percepção de corrupção (Corruption Perception Index - CPI)

Outras: 

In [1]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
import numpy as np

import statsmodels.api as sm1 #Usa OLS com X como matriz de planejamento
import statsmodels.formula.api as sm  #Usa OLS como no R

print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/AndreEjz/Desktop/CDP3


In [2]:
d1 = pd.read_excel('Dados.xlsx', index_col=0)
d2 = pd.read_excel('Corrupcao.xlsx', index_col=0)

In [5]:
d2.head()

,CPI
Paises,
Afghanistan,1.4
Albania,3.3
Algeria,2.9
Angola,1.9
Argentina,2.9


In [4]:
d1.head()

,Internet,Celular,Renda,IDH,Saneamento,Servicos
Países,,,,,,
Abkhazia,NaN,NaN,NaN,NaN,NaN,47.607657
Afghanistan,3.246306,34.338687,NaN,0.387,37.0,60.239255
Akrotiri and Dhekelia,NaN,NaN,NaN,NaN,NaN,30.960967
Albania,41.200000,77.167390,1857.352947,0.734,94.0,NaN
Algeria,11.230000,93.647115,2192.703976,0.691,95.0,NaN


In [6]:
dados = d2.join(d1, how='outer')
dados.head()

,CPI,Internet,Celular,Renda,IDH,Saneamento,Servicos
Abkhazia,NaN,NaN,NaN,NaN,NaN,NaN,47.607657
Afghanistan,1.4,3.246306,34.338687,NaN,0.387,37.0,60.239255
Akrotiri and Dhekelia,NaN,NaN,NaN,NaN,NaN,NaN,30.960967
Albania,3.3,41.200000,77.167390,1857.352947,0.734,94.0,NaN
Algeria,2.9,11.230000,93.647115,2192.703976,0.691,95.0,NaN


In [7]:
dados=dados[~np.isnan(dados.CPI)]
dados=dados[~np.isnan(dados.Internet)]
dados=dados[~np.isnan(dados.Celular)]
dados=dados[~np.isnan(dados.Renda)]
dados=dados[~np.isnan(dados.IDH)]
dados=dados[~np.isnan(dados.Saneamento)]
dados=dados[~np.isnan(dados.Servicos)]

dados.shape

(83, 7)